In [ ]:
import random
import time
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc
from datetime import datetime
import uuid

# Configura el navegador con undetected_chromedriver y un User-Agent aleatorio
options = uc.ChromeOptions()
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36",
]
options.add_argument(f"user-agent={random.choice(user_agents)}")
browser = uc.Chrome(options=options)

# URL inicial: Página de listados de Madrid
base_url = "https://www.pisos.com/venta/pisos-madrid/"
browser.get(base_url)

# Manejar las cookies
try:
    WebDriverWait(browser, 20).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="didomi-notice-agree-button"]'))
    ).click()
except Exception:
    print("No se encontró el botón de cookies o ya fue aceptado.")
time.sleep(5)  # Añadir pausa después de aceptar cookies

# Lista para almacenar los datos
all_data = []
inmueble_counter = 1  # Contador de inmuebles
max_inmuebles = 9683  # Número máximo de inmuebles que queremos recolectar

# Función para extraer datos de un inmueble dado su enlace
def extraer_datos_inmueble():
    try:
        WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.TAG_NAME, 'body')))
        html_inmueble = browser.page_source
        soup_inmueble = bs(html_inmueble, 'lxml')
        url_actual = browser.current_url
        # Extraer datos del inmueble
        try:
            # Descripción del inmueble (H1)
            descripcion = soup_inmueble.find('h1').text.strip() if soup_inmueble.find('h1') else "Descripción no disponible"

            # Localización en el primer <p> después del <h1> (ejemplo: Centro (Galapagar))
            localizacion = soup_inmueble.find('p').text.strip() if soup_inmueble.find('p') else "Localización no disponible"

            # Precio
            precio_element = soup_inmueble.find('div', {'class': 'price__value jsPriceValue'})
            precio = precio_element.text.split(' ')[0] if precio_element else "N/A"

            # Superficie construida
            superficie_element = soup_inmueble.find('span', {'class': 'features__value'})
            superficie_construida = superficie_element.text.split(' ')[0] if superficie_element else "N/A"

            # Última actualización
            ultima_actualizacion_element = soup_inmueble.find('p', {'class': 'last-update__date'})
            ultima_actualizacion = ultima_actualizacion_element.text if ultima_actualizacion_element else "N/A"

            # Características
            c1 = soup_inmueble.find('div', {'class': 'features__content'})
            features_list = []
            if c1:
                features = c1.find_all('div', {'class': 'features__feature'})
                for feature in features:
                    label = feature.find('span', {'class': 'features__label'})
                    value = feature.find('span', {'class': 'features__value'})
                    if label and value:
                        features_list.append((label.get_text(strip=True), value.get_text(strip=True)))
            else:
                features_list = "N/A"

            # Certificado energético
            energy_certificate = soup_inmueble.find('div', {'class': 'details__block energy-certificate'})
            if energy_certificate:
                consumo_element = energy_certificate.find('div', {'class': 'energy-certificate__data'})
                consumo = consumo_element.find_all('span')[1].get_text(strip=True) if consumo_element else "N/A"
                
                emisiones_element = energy_certificate.find_all('div', {'class': 'energy-certificate__data'})
                emisiones = emisiones_element[1].find_all('span')[1].get_text(strip=True) if len(emisiones_element) > 1 else "N/A"
            else:
                consumo = "N/A"
                emisiones = "N/A"

            # Agregar el timestamp de scrapeo y un identificador único
            data = {
                'id': str(uuid.uuid4()),  # ID único para cada inmueble
                'Descripción': descripcion,
                'Localización': localizacion,
                'Enlace': url_actual,
                'Precio': precio,
                'Superficie Construida': superficie_construida,
                'Última Actualización': ultima_actualizacion,
                'Consumo Energético': consumo,
                'Emisiones CO2': emisiones,
                'Características': features_list,
                'Tipo de operación': 'compra',
                'timestamp_scrapeo': datetime.now().isoformat()  # Timestamp de scrapeo
            }

            return data

        except Exception as e:
            print(f"Error al extraer datos: {e}")
            return None

    except Exception as e:
        print(f"Error al cargar la página del inmueble: {e}")
        return None

# Función para avanzar al siguiente inmueble
    
def ir_al_siguiente_inmueble():
    try:
        # Intentar con el primer selector
        siguiente_boton = WebDriverWait(browser, 20).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.navigation__link.navigation__link--next'))
        )
        siguiente_boton.click()
        time.sleep(2)  # Pausa para cargar el siguiente inmueble
        return True
    except Exception as e1:
        print(f"No se encontró el botón de 'Siguiente' con el primer selector: {e1}")
        try:
            # Intentar con el segundo selector si el primero falla
            siguiente_boton = WebDriverWait(browser, 20).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, '.navigation-arrow--next'))
            )
            siguiente_boton.click()
            time.sleep(2)  # Pausa para cargar el siguiente inmueble
            return True
        except Exception as e2:
            print(f"No se encontró el botón de 'Siguiente' con el segundo selector: {e2}")
            return False

# Iniciar en el primer inmueble
try:
    primer_inmueble = WebDriverWait(browser, 20).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, '.ad-preview'))
    )
    primer_inmueble.click()
    time.sleep(2)  # Pausa para cargar la página del primer inmueble
except Exception as e:
    print(f"Error al abrir el primer inmueble: {e}")

# Bucle para extraer datos de cada inmueble hasta alcanzar el máximo o agotar los inmuebles
while len(all_data) < max_inmuebles:
    data = extraer_datos_inmueble()
    if data:
        all_data.append(data)
        print(f"Inmueble {inmueble_counter}: {data}")
        inmueble_counter += 1

    # Intentar ir al siguiente inmueble
    if not ir_al_siguiente_inmueble():
        continue  

# Convertir los datos en un DataFrame de pandas y guardarlos
df = pd.DataFrame(all_data)
print(df)

# Opcional: guarda el DataFrame en un archivo CSV
df.to_csv('inmuebles_venta.csv', index=False)

# Cierra el navegador
browser.quit()


Inmueble 1: {'id': 'f6ef94f7-0b1c-44e3-ae35-0e1173fe3ae7', 'Descripción': 'Chalet pareado en venta en Calle Cantabria, cerca de Calle Galicia', 'Localización': 'Serranillos del Valle', 'Enlace': 'https://www.pisos.com/comprar/chalet_pareado-serranillos_del_valle_centro_urbano-45091667204_993816/', 'Precio': '374.000', 'Superficie Construida': '264', 'Última Actualización': 'Anuncio actualizado el 06/11/2024', 'Consumo Energético': 'N/A', 'Emisiones CO2': 'N/A', 'Características': [('Superficie construida:', '264 m²'), ('Superficie útil:', '264 m²'), ('Superficie solar:', '457 m²'), ('Habitaciones:', '5'), ('Baños:', '4'), ('Antigüedad:', 'Entre 20 y 30 años'), ('Conservación:', 'En buen estado'), ('Referencia:', '4993816-003274')], 'Tipo de operación': 'compra', 'timestamp_scrapeo': '2024-11-06T09:29:36.208047'}
Inmueble 2: {'id': '13947572-4b78-49df-a7ba-33b40b837528', 'Descripción': 'Piso en venta en Calle de Pablo Picasso, cerca de Calle de la Constitución', 'Localización': 'Centro 

KeyboardInterrupt: 

In [4]:
df

,id,Descripción,Localización,Enlace,Precio,Superficie Construida,Última Actualización,Consumo Energético,Emisiones CO2,Características,Tipo de operación,timestamp_scrapeo
0,bfcba5e4-5958-4445-9de7-0ab4df732b8c,Chalet en venta en Boadilla del Monte - Parque...,Parque de Boadilla (Boadilla del Monte),https://www.pisos.com/comprar/chalet-parque_de...,1.795.000,415,Anuncio actualizado el 05/11/2024,Consumo:104 kWh/m² año,Emisiones:53 Kg CO₂/m² año,"[(Superficie construida:, 415 m²), (Superficie...",compra,2024-11-05T23:52:42.741334
1,a0f3e8d5-ca6f-45c5-a5d4-2d3d6b7bbd22,Casa pareada en venta en Cobeña,Cobeña,https://www.pisos.com/comprar/casa_pareada-cob...,670.000,360,Anuncio actualizado el 05/11/2024,Consumo:104 kWh/m² año,Emisiones:53 Kg CO₂/m² año,"[(Superficie construida:, 360 m²), (Superficie...",compra,2024-11-05T23:52:45.723079
2,2efd9d2c-cdc9-45a6-91eb-6173f0e94ac7,"Piso en venta en Calle Joaquim Blume, Número 14",Zona Suroeste (Torrejón de Ardoz),https://www.pisos.com/comprar/piso-zona_suroes...,254.900,103,Anuncio actualizado el 02/11/2024,N/A,N/A,"[(Superficie construida:, 103 m²), (Superficie...",compra,2024-11-05T23:52:48.291304
3,15c5ec6e-b28e-42d1-8fb9-497fa3158469,"Piso en venta en Avenida de Concha Espina, cer...",Hispanoamérica (Distrito Chamartín. Madrid Cap...,https://www.pisos.com/comprar/piso-chamartin_h...,950.000,141,Anuncio actualizado el 05/11/2024,Consumo:104 kWh/m² año,Emisiones:56 Kg CO₂/m² año,"[(Superficie construida:, 141 m²), (Habitacion...",compra,2024-11-05T23:52:50.925002
4,a0676ed8-fa97-44ac-b242-aa947e8a2016,Piso en venta en Calle del Duratón,Lucero (Distrito Latina. Madrid Capital),https://www.pisos.com/comprar/piso-lucero28011...,210.000,65,Anuncio actualizado el 17/10/2024,N/A,N/A,"[(Superficie construida:, 65 m²), (Superficie ...",compra,2024-11-05T23:52:53.676227
...,...,...,...,...,...,...,...,...,...,...,...,...
9678,ed977a06-35d6-4d05-bed2-b3a7afc07290,"Casas y pisos en Aravaca, Distrito Moncloa-Ara...",Sé el primero en enterarte,https://www.pisos.com/comprar/piso-moncloa_ara...,N/A,N/A,N/A,N/A,N/A,N/A,compra,2024-11-06T08:53:23.696275
9679,9796437d-4616-43fd-a0c5-590a338d2678,"Casas y pisos en Aravaca, Distrito Moncloa-Ara...",Sé el primero en enterarte,https://www.pisos.com/comprar/piso-moncloa_ara...,N/A,N/A,N/A,N/A,N/A,N/A,compra,2024-11-06T08:53:44.061304
9680,f5c76a54-c3bf-445d-ba23-4b516989214d,"Casas y pisos en Aravaca, Distrito Moncloa-Ara...",Sé el primero en enterarte,https://www.pisos.com/comprar/piso-moncloa_ara...,N/A,N/A,N/A,N/A,N/A,N/A,compra,2024-11-06T08:54:04.448888
9681,99ddc50b-86a9-4e62-8119-f8aa293ca866,"Casas y pisos en Aravaca, Distrito Moncloa-Ara...",Sé el primero en enterarte,https://www.pisos.com/comprar/piso-moncloa_ara...,N/A,N/A,N/A,N/A,N/A,N/A,compra,2024-11-06T08:54:24.756995
